In [1]:
import spnutils
import embedding
import crv

In [2]:
x = polygen(GF(2), 'x')
F4 = GF(2**4, name=x, modulus=x**4 + x + 1)

In [3]:
PHOTON_S4 = [0xc, 0x5, 0x6, 0xb, 0x9, 0x0, 0xa, 0xd, 0x3, 0xe, 0xf, 0x8, 0x4, 0x7, 0x1, 0x2]

### Polynomial Interpolation for PHOTON 4-bit S-box

In [4]:
c = spnutils.interpolate(F4, [spnutils.natural_encoding(F4, i) for i in range(16)], [spnutils.natural_encoding(F4, x) for x in PHOTON_S4])
print(c)

(x^3 + x^2, 0, x^2 + x + 1, x^2 + x + 1, x^3 + x^2 + x, x^3 + x, x^3 + x^2, x^2, x^2 + x + 1, x^3 + 1, x^3 + 1, x^3 + x^2 + x, x^3 + x^2, x^3 + x^2 + 1, x^3 + x^2 + 1, 0)


In [5]:
print(' + '.join([f'0x{spnutils.natural_encoding_to_int(ci):x} z^{i}' for i,ci in enumerate(c)]))

0xc z^0 + 0x0 z^1 + 0x7 z^2 + 0x7 z^3 + 0xe z^4 + 0xa z^5 + 0xc z^6 + 0x4 z^7 + 0x7 z^8 + 0x9 z^9 + 0x9 z^10 + 0xe z^11 + 0xc z^12 + 0xd z^13 + 0xd z^14 + 0x0 z^15


### Addition Chain

In [6]:
res = spnutils.shallow_dense_addition_chain(values=[i for i,ci in enumerate(c) if ci != 0], n=4, doubles=[1,3,5,7])
print(f'Requires {res.required_levels()} rounds')
muls, squares, bd = res.required_additions()
print(f'muls: {muls+squares}, free squares: {len(res._frees)}, bits: {4 * bd} in {res.required_levels()} rounds')
res.print_levels()

Requires 4 rounds
muls: 3, free squares: 10, bits: 16 in 4 rounds
Round 1 [free]: 2 = 1 + 1, 4 = 2 + 2, 8 = 4 + 4
Round 1: 
Round 2 [free]: 
Round 2: 3 = 1 + 2, 5 = 1 + 4
Round 3 [free]: 6 = 3 + 3, 12 = 6 + 6, 9 = 12 + 12, 10 = 5 + 5
Round 3: 7 = 1 + 6
Round 4 [free]: 14 = 7 + 7, 13 = 14 + 14, 11 = 13 + 13
Round 4: 


In [7]:
def serial_mixing_matrix(F, zi):
    """ Returns the mixing matrix of the step "MixColumnsSerial" of the PHOTON permutation (https://eprint.iacr.org/2011/609) """
    d = len(zi)
    A = matrix(F, d, d, 0)
    for i in range(d-1):
        A[i,i+1] = F(1)
    for i in range(d):
        A[d-1,i] = zi[i]
    for i in range(d):
        A *= A
    return A

In [8]:
A_5 = serial_mixing_matrix(F4, [spnutils.natural_encoding(F4, x) for x in [1,2,9,9,2]])
for i in range(5):
    c = ','.join(f'0x{spnutils.natural_encoding_to_int(A_5[i,j]):x}' for j in range(5))
    print(f'[{c}]')

[0x0,0x0,0x1,0x0,0x0]
[0x0,0x0,0x0,0x1,0x0]
[0x0,0x0,0x0,0x0,0x1]
[0x1,0x2,0x9,0x9,0x2]
[0x2,0x5,0x3,0x8,0xd]


### Find Embedding for MP-SPDZ implementation

In [9]:
Y = polygen(GF(2), 'Y')
F40 = GF(2**40, name='Y', modulus=Y^40 + Y^20 + Y^15 + Y^10 + 1) # the target field used in MP-SPDZ
embedding, _ = embedding.find_min_cost_embedding(F4, F40)
print(embedding)

Embedding from Finite Field in x of size 2^4 to Finite Field in Y of size 2^40 via Y^35 + Y^20 + Y^15


#### Code to embed coefficients x0,x1,x2,x3 of F4 into F40 (y0,y5,y10,y15,y20,y30,y35)

In [10]:
embedding.print_impl_forward()

y0 = x0
y5 = x2 + x3
y10 = x2
y15 = x1 + x2 + x3
y20 = x1
y30 = x2
y35 = x1


#### Code to reverse the embedding from F40 to F4

In [11]:
embedding.print_impl_backward()

x0 = y0
x1 = y5 + y15
x2 = y10
x3 = y5 + y10


### CRV Polynomial Decomposition

In [12]:
# this code generates a random q polynomial
# instance = crv.CRV(F4, [0,1,3])
# instance.find_q_polynomials()

# this code uses the q polynomial from the paper
instance = crv.CRV(F4, [0,1,3])
q0 = [spnutils.natural_encoding(F4, c) for c in [0x0,0x6,0x4,0xd,0x3,0x4,0x8,0xb,0x8]]
instance.set_q_polynomials([q0])

In [13]:
res = instance.polynomial_decomposition(PHOTON_S4)

In [14]:
for q in res.q:
    print(','.join(f'0x{spnutils.natural_encoding_to_int(c):x} z^{i}' for i,c in enumerate(q.coefficients(sparse=False)) if c != 0))

0x6 z^1,0x4 z^2,0xd z^3,0x3 z^4,0x4 z^6,0x8 z^8,0xb z^9,0x8 z^12


In [15]:
for p in res.p:
    pi = [F4(list(c)[0]) for c in p]
    print(','.join(f'0x{spnutils.natural_encoding_to_int(c):x} z^{i}' for i,c in enumerate(pi) if c != 0))

0xd z^0,0x7 z^1,0xc z^2,0x7 z^3,0xe z^5,0xb z^6,0x2 z^7,0x1 z^8
0xc z^0,0xc z^1,0xd z^2,0xb z^4,0x7 z^5,0xd z^6


### Code for free squaring in F4

In [16]:
spnutils.gf_squaring(F4)

[1 0 1 0]
[0 0 1 0]
[0 1 0 1]
[0 0 0 1]

In [17]:
spnutils.print_cmul_code(F4, [2,3,5,8,9,10,11,12,13,14,15])

2: 
    y0 = x3
    y1 = x0 + x3
    y2 = x1
    y3 = x2
3: 
    y0 = x0 + x3
    y1 = x0 + x1 + x3
    y2 = x1 + x2
    y3 = x2 + x3
5: 
    y0 = x0 + x2
    y1 = x1 + x2 + x3
    y2 = x0 + x2 + x3
    y3 = x1 + x3
8: 
    y0 = x1
    y1 = x1 + x2
    y2 = x2 + x3
    y3 = x0 + x3
9: 
    y0 = x0 + x1
    y1 = x2
    y2 = x3
    y3 = x0
10: 
    y0 = x1 + x3
    y1 = x0 + x1 + x2 + x3
    y2 = x1 + x2 + x3
    y3 = x0 + x2 + x3
11: 
    y0 = x0 + x1 + x3
    y1 = x0 + x2 + x3
    y2 = x1 + x3
    y3 = x0 + x2
12: 
    y0 = x1 + x2
    y1 = x1 + x3
    y2 = x0 + x2
    y3 = x0 + x1 + x3
13: 
    y0 = x0 + x1 + x2
    y1 = x3
    y2 = x0
    y3 = x0 + x1
14: 
    y0 = x1 + x2 + x3
    y1 = x0 + x1
    y2 = x0 + x1 + x2
    y3 = x0 + x1 + x2 + x3
15: 
    y0 = x0 + x1 + x2 + x3
    y1 = x0
    y2 = x0 + x1
    y3 = x0 + x1 + x2


In [18]:
F8 = GF(2**8, name=x, modulus=x**8 + x**4 + x**3 + x + 1)

In [19]:
cmuls = [[2,3,1,2,1,4],
    [8, 14, 7, 9, 6, 17],
    [34, 59, 31, 37, 24, 66],
    [132, 228, 121, 155, 103, 11],
    [22, 153, 239, 111, 144, 75],
    [150, 203, 210, 121, 36, 167]]
cmuls = set(x for l in cmuls for x in l)
cmuls = sorted(cmuls)
print(cmuls)

[1, 2, 3, 4, 6, 7, 8, 9, 11, 14, 17, 22, 24, 31, 34, 36, 37, 59, 66, 75, 103, 111, 121, 132, 144, 150, 153, 155, 167, 203, 210, 228, 239]


In [20]:
spnutils.print_cmul_code(F8, cmuls)

1: 
    y0 = x0
    y1 = x1
    y2 = x2
    y3 = x3
    y4 = x4
    y5 = x5
    y6 = x6
    y7 = x7
2: 
    y0 = x7
    y1 = x0 + x7
    y2 = x1
    y3 = x2 + x7
    y4 = x3 + x7
    y5 = x4
    y6 = x5
    y7 = x6
3: 
    y0 = x0 + x7
    y1 = x0 + x1 + x7
    y2 = x1 + x2
    y3 = x2 + x3 + x7
    y4 = x3 + x4 + x7
    y5 = x4 + x5
    y6 = x5 + x6
    y7 = x6 + x7
4: 
    y0 = x6
    y1 = x6 + x7
    y2 = x0 + x7
    y3 = x1 + x6
    y4 = x2 + x6 + x7
    y5 = x3 + x7
    y6 = x4
    y7 = x5
6: 
    y0 = x6 + x7
    y1 = x0 + x6
    y2 = x0 + x1 + x7
    y3 = x1 + x2 + x6 + x7
    y4 = x2 + x3 + x6
    y5 = x3 + x4 + x7
    y6 = x4 + x5
    y7 = x5 + x6
7: 
    y0 = x0 + x6 + x7
    y1 = x0 + x1 + x6
    y2 = x0 + x1 + x2 + x7
    y3 = x1 + x2 + x3 + x6 + x7
    y4 = x2 + x3 + x4 + x6
    y5 = x3 + x4 + x5 + x7
    y6 = x4 + x5 + x6
    y7 = x5 + x6 + x7
8: 
    y0 = x5
    y1 = x5 + x6
    y2 = x6 + x7
    y3 = x0 + x5 + x7
    y4 = x1 + x5 + x6
    y5 = x2 + x6 + x7
    y6 = x3 + 